In [3]:
import laser_lib
import numpy as np
import math
import time
import random
queue = laser_lib.DacQueue()

Found  1 Helios DACs


### Test electronic vs mechanical rolling shutter effect

In [ ]:
T = 1000
queue.dac_rate = T
counter = 0

arr_pos = np.zeros((T, 2))
arr_col = np.zeros((T, 3))
for t in range(T):
    r_bit = (counter >> 2) & 1
    g_bit = (counter >> 1) & 1
    b_bit = (counter >> 0) & 1
    counter += 1
    print(r_bit, g_bit, b_bit)
    arr_col[t, :] = np.array([r_bit, g_bit, b_bit])
while(True):
    queue.submit(arr_pos, arr_col)

In [7]:
T = 100
queue.dac_rate = 50000

arr_pos = np.ones((T, 2))

color = 0
while(True):
    arr_col = np.zeros((T, 3))
    arr_col[:, color] = 1
    color += 1 
    color = color % 3
    queue.submit(arr_pos, arr_col)

KeyboardInterrupt: 

### Test delay between sending a shutter release signal and the shutter front curtain (electronic)

In [25]:
# bands = list(np.arange(1, 8))
bands = list(np.arange(0, 4))
unique_orders = set()

In [26]:
for i in range(10000):
    random.shuffle(bands)
    # print('Cand: ', bands)
    unique_orders.add(tuple(bands))
unique_orders = list(unique_orders)

In [32]:
print(len(unique_orders))
print(unique_orders)
map = {0: 'BLK', 1: 'BLU', 2: 'GRN', 3: 'TRQ'}
for ord in unique_orders:
    # print([bin(c) for c in ord])
    print([map[c] for c in ord])

24
[(3, 2, 1, 0), (1, 0, 3, 2), (1, 3, 2, 0), (3, 2, 0, 1), (2, 1, 0, 3), (2, 0, 3, 1), (1, 3, 0, 2), (0, 2, 3, 1), (2, 3, 1, 0), (1, 0, 2, 3), (1, 2, 3, 0), (0, 3, 2, 1), (2, 0, 1, 3), (3, 1, 2, 0), (0, 2, 1, 3), (3, 0, 2, 1), (2, 3, 0, 1), (0, 3, 1, 2), (0, 1, 2, 3), (0, 1, 3, 2), (3, 0, 1, 2), (2, 1, 3, 0), (3, 1, 0, 2), (1, 2, 0, 3)]
['TRQ', 'GRN', 'BLU', 'BLK']
['BLU', 'BLK', 'TRQ', 'GRN']
['BLU', 'TRQ', 'GRN', 'BLK']
['TRQ', 'GRN', 'BLK', 'BLU']
['GRN', 'BLU', 'BLK', 'TRQ']
['GRN', 'BLK', 'TRQ', 'BLU']
['BLU', 'TRQ', 'BLK', 'GRN']
['BLK', 'GRN', 'TRQ', 'BLU']
['GRN', 'TRQ', 'BLU', 'BLK']
['BLU', 'BLK', 'GRN', 'TRQ']
['BLU', 'GRN', 'TRQ', 'BLK']
['BLK', 'TRQ', 'GRN', 'BLU']
['GRN', 'BLK', 'BLU', 'TRQ']
['TRQ', 'BLU', 'GRN', 'BLK']
['BLK', 'GRN', 'BLU', 'TRQ']
['TRQ', 'BLK', 'GRN', 'BLU']
['GRN', 'TRQ', 'BLK', 'BLU']
['BLK', 'TRQ', 'BLU', 'GRN']
['BLK', 'BLU', 'GRN', 'TRQ']
['BLK', 'BLU', 'TRQ', 'GRN']
['TRQ', 'BLK', 'BLU', 'GRN']
['GRN', 'BLU', 'TRQ', 'BLK']
['TRQ', 'BLU', 'BLK', 

In [12]:
T = 400
queue.dac_rate = 50000
t = 0

arr_pos = np.zeros((T, 2))
b = 0
while(True):
    for idx, band in enumerate(unique_orders[b % 6]): 
        arr_col = np.zeros((T, 3))
        r_bit = (band >> 2) & 1
        g_bit = (band >> 1) & 1
        b_bit = (band >> 0) & 1
        arr_col[:, :] = np.array([r_bit, g_bit, b_bit])
        queue.submit(arr_pos, arr_col)

KeyboardInterrupt: 

In [29]:
T = 400
queue.dac_rate = 50000
t = 0

intensity_hold = -3.0 / 5.0 
intensity_release = 0.0 / 5.0 
arr_col_hold = np.zeros((T, 3))
arr_pos_hold = np.zeros((T, 2))
arr_pos_hold[:, :] = intensity_hold
arr_pos_release = np.zeros((T, 2))
arr_pos_release[:, :] = intensity_release

wait_seconds = 2
wait_steps = int(wait_seconds / (T / queue.dac_rate))
for w in range(wait_steps):
    print('Waiting {0} steps'.format(w), end='\r')
    queue.submit(arr_pos_hold, arr_col_hold)

print('\n\nReleasing Shutter\n\n')
for idx, sequence in enumerate(unique_orders): 
    print('Sequence {0} / {1}'.format(idx, len(unique_orders)), end='\r')
    for band in sequence:
        arr_col = np.zeros((T, 3))
        r_bit = (band >> 2) & 1
        g_bit = (band >> 1) & 1
        b_bit = (band >> 0) & 1
        arr_col[:, :] = np.array([r_bit, g_bit, b_bit])
        queue.submit(arr_pos_release, arr_col)

Waiting 249 steps

Releasing Shutter


